# Part-of-Speech tagging tutorial with the Keras Deep Learning library
## https://becominghuman.ai/part-of-speech-tagging-tutorial-with-the-keras-deep-learning-library-d7f93fa05537

## Ensuring reproducibility

In [1]:
import numpy as np
CUSTOM_SEED = 42
np.random.seed(CUSTOM_SEED)

## Getting an annotated corpus

In [10]:
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

In [11]:
from nltk.corpus import treebank
sentences = treebank.tagged_sents(tagset='universal')

In [12]:
import random
print(random.choice(sentences))

[('The', 'DET'), ('screen', 'NOUN'), ('fills', 'VERB'), ('with', 'ADP'), ('a', 'DET'), ('small', 'ADJ'), (',', '.'), ('tight', 'ADJ'), ('facial', 'ADJ'), ('shot', 'NOUN'), ('of', 'ADP'), ('David', 'NOUN'), ('Dinkins', 'NOUN'), (',', '.'), ('Democratic', 'ADJ'), ('candidate', 'NOUN'), ('for', 'ADP'), ('mayor', 'NOUN'), ('of', 'ADP'), ('New', 'NOUN'), ('York', 'NOUN'), ('City', 'NOUN'), ('.', '.')]


In [13]:
tags = set([
    tag for sentence in treebank.tagged_sents() 
    for _, tag in sentence
])
print('nb_tags: %sntags: %s' % (len(tags), tags))

nb_tags: 46ntags: {'LS', 'EX', "''", 'TO', 'RBS', 'JJS', '``', 'NN', 'DT', 'JJR', 'RB', 'VBG', 'WRB', 'PRP', 'WP$', 'VBN', 'POS', 'VBD', 'WDT', 'MD', 'NNPS', 'SYM', '-NONE-', 'CC', '-LRB-', 'IN', '.', 'VBZ', 'RP', 'JJ', 'NNS', ':', 'VBP', 'NNP', 'PDT', 'WP', 'UH', 'CD', 'VB', 'PRP$', ',', '#', 'RBR', 'FW', '-RRB-', '$'}


## Datasets preprocessing for supervised learning

In [24]:
train_test_cutoff = int(.80 * len(sentences)) 
training_sentences = sentences[:train_test_cutoff]
testing_sentences = sentences[train_test_cutoff:]
train_val_cutoff = int(.25 * len(training_sentences))
validation_sentences = training_sentences[:train_val_cutoff]
training_sentences = training_sentences[train_val_cutoff:]

## Feature engineering

In [25]:
def add_basic_features(sentence_terms, index):
    """ Compute some very basic word features.
        :param sentence_terms: [w1, w2, ...] 
        :type sentence_terms: list
        :param index: the index of the word 
        :type index: int
        :return: dict containing features
        :rtype: dict
    """
    term = sentence_terms[index]
    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]
    }

In [26]:
def untag(tagged_sentence):
    """ 
    Remove the tag for each tagged term.
    :param tagged_sentence: a POS tagged sentence
    :type tagged_sentence: list
    :return: a list of tags
    :rtype: list of strings
    """
    return [w for w, _ in tagged_sentence]

def transform_to_dataset(tagged_sentences):
    """
    Split tagged sentences to X and y datasets and append some basic features.
    :param tagged_sentences: a list of POS tagged sentences
    :param tagged_sentences: list of list of tuples (term_i, tag_i)
    :return: 
    """
    X, y = [], []
    for pos_tags in tagged_sentences:
        for index, (term, class_) in enumerate(pos_tags):
            # Add basic NLP features for each sentence term
            X.append(add_basic_features(untag(pos_tags), index))
            y.append(class_)
    return X, y

In [27]:
X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(testing_sentences)
X_val, y_val = transform_to_dataset(validation_sentences)

## Features encoding

In [29]:
from sklearn.feature_extraction import DictVectorizer

# Fit our DictVectorizer with our set of features
dict_vectorizer = DictVectorizer(sparse=False)
print(type(X_train), type(X_test), type(X_val))
dict_vectorizer.fit(X_train + X_test + X_val)

# Convert dict features to vectors
X_train = dict_vectorizer.transform(X_train)
X_test = dict_vectorizer.transform(X_test)
X_val = dict_vectorizer.transform(X_val)

<class 'numpy.ndarray'> <class 'list'> <class 'list'>


ValueError: operands could not be broadcast together with shapes (61107,44232) (20039,) 

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Fit LabelEncoder with our list of classes
label_encoder = LabelEncoder()
label_encoder.fit(y_train + y_test + y_val)

# Encode class values as integers
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)
y_val = label_encoder.transform(y_val)

In [ ]:
# Convert integers to dummy variables (one hot encoded)
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_val = np_utils.to_categorical(y_val)

## Building a Keras model

In [30]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

def build_model(input_dim, hidden_neurons, output_dim):
    """
    Construct, compile and return a Keras model which will be used to fit/predict
    """
    model = Sequential([
        Dense(hidden_neurons, input_dim=input_dim),
        Activation('relu'),
        Dropout(0.2),
        Dense(hidden_neurons),
        Activation('relu'),
        Dropout(0.2),
        Dense(output_dim, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

ModuleNotFoundError: No module named 'keras'